##Necessary imports

In [ ]:
!pip3 install newspaper3k

import requests
from bs4 import BeautifulSoup
from newspaper import Article
import re

  Using cached https://files.pythonhosted.org/packages/d7/b9/51afecb35bb61b188a4b44868001de348a0e8134b4dfa00ffc191567c4b9/newspaper3k-0.2.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1c/21/faf1bac028662cc8adb2b5ef7a6f3999a765baa2835331df365289b0ca56/feedparser-6.0.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7e/62/b6acd3129c5615b9860e670df07fd55b76175b63e6b7f68282c7cad38e9e/tldextract-3.1.0-py2.py3-none-any.whl
Processing /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02/tinysegmenter-0.3-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3b/d4/3b5c17f00cce85b9a1e6f91096e1cc8e8ede2e1be8e96b87ce1ed09e92c5/cssselect-1.1.0-py2.py3-none-any.whl
Processing /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed/feedfinder2-0.0.4-cp37-none-any.whl
Processing /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34/jieba3k-0.3

**n** - number of next pages from single web page




In [ ]:
n = 100

I have picked three online fashion magazines **ELLE**, **GLAMOUR** and **AVANTI** and scrapped their web pages for fashion articles.

##ELLE

In [ ]:
elle = "https://www.elle.pl/moda?page=2"
elle_format = "https://www.elle.pl/moda{}"
home_format = "https://www.elle.pl{}"

In [ ]:

elle_links = []
elle_text = ''

page = requests.get(elle)
soup = BeautifulSoup(page.content, 'html.parser')

for i in range(n):

  links = [div.a for div in
        soup.findAll('div', attrs={'class' : " col-12 col-md-6 col-lg-6 col-xl-6 box-all-link d-flex flex-column "})]
  for link in links:

      elle_links.append(elle_format.format(link['href']))
      elle_article = Article(elle_format.format(link['href']), language="pl")
      elle_article.download()

      try:
        elle_article.parse()
        elle_text += elle_article.text.strip()
      except :
        pass

  next_page = [span.a for span in
        soup.findAll('span', attrs={'class' : "next"})]
  next_link = requests.get(home_format.format(next_page[0]['href']))
  soup = BeautifulSoup(next_link.content, 'html.parser')

##GLAMOUR

In [ ]:
glamour = 'https://www.glamour.pl/tag/moda'
glamour_format = 'https://www.glamour.pl{}'

In [ ]:
glamour_links = []
glamour_text = ''


page = requests.get(glamour)
soup = BeautifulSoup(page.content, 'html.parser')

for i in range(n):
  for link in soup.findAll('a', {'class': "d-flex flex-column box-all-link"}):
    if re.findall('/artykul/', link.get('href')) != []:
      glamour_links.append(glamour_format.format(link.get('href')))
      glamour_article = Article(glamour_format.format(link['href']), language="pl")
      glamour_article.download()
      try:
        glamour_article.parse()
        glamour_text += glamour_article.text.strip()
      except :
        pass

  next_link = soup.findAll('span', attrs={'class': "next"})[0].findAll('a')[0].get('href')
  next_page = requests.get(glamour_format.format(next_link))
  soup = BeautifulSoup(next_page.content, 'html.parser')



##AVANTI

In [ ]:
avanti = 'https://avanti24.pl/Magazyn/0,150441.html'
avanti_format = 'https://avanti24.pl{}'

In [ ]:
avanti_links = []
avanti_text = ''


page = requests.get(avanti)
soup = BeautifulSoup(page.content, 'html.parser')

for i in range(n):
  links = [div.a for div in soup.findAll('div', attrs={'class' : "imgw"})]
  for link in links:
    if link != None and re.findall('/Magazyn/', link['href']) != [] and link['href'] not in avanti_links:
      avanti_links.append(link['href'])
      avanti_article = Article(link['href'], language="pl")
      avanti_article.download()
      try:
        avanti_article.parse()
        avanti_text += avanti_article.text.strip()
      except :
        pass

  next_page = requests.get(avanti_format.format(soup.findAll('a', {'class': 'next'})[0]['href']))
  soup = BeautifulSoup(next_page.content, 'html.parser')


In [ ]:
data = elle_text + glamour_text + avanti_text # combined dataset

In [ ]:
len(set(data.split())) # number of tokens in data

In [ ]:
path = '/content/drive/MyDrive/'
textfile = open("fashion_corpus.txt", 'w')
textfile.write(data)
textfile.close()